# Demo for using `hifir4py` #
In this example, we show how to use `hifir4py` HIFIR preconditioner in its multilevel triangular solve and matrix-vector multiplication, which are the core operations in applying a preconditioner.

In [1]:
import numpy as np
from scipy.io import loadmat
from hifir4py import *

In [2]:
# load the MATFILE from scipy.io
f = loadmat("demo_inputs/data.mat")
A = f["A"]
b = f["b"].reshape(-1)

Let's show some basic information of the system, including shape, nnz, and leading block symmetry

In [3]:
# A is scipy.sparse.csr_matrix
print("The system shape is {}, where the nnz is {}".format(A.shape, A.nnz))

The system shape is (2990, 2990), where the nnz is 44632


## Factorize HIF ##

Now, let's build the preconditioenr $\boldsymbol{M}$ with more aggressive options, i.e. `droptol` for L and U factors is 1e-2, `condest` for L, U, and D is 5, and $\alpha$ for L and U is 3.

In [5]:
M = HIF(
    A,
    tau_L=0.01,
    tau_U=0.01,
    kappa=5.0,
    kappa_d=5.0,
    alpha_L=3.0,
    alpha_U=3.0,
)


|           HIF: Hybrid Incomplete Factorization                      |
|                                                                     |
| HIF is a package for computing hybrid incomplete factorization      |
| with near linear time complexity.                                   |
-----------------------------------------------------------------------

 > Package information:

	* Copyright (C) 2019--2021 NumGeom Group at Stony Brook University
	* Version: 0.1.0


Params (control parameters) are:

tau_L                         0.010000
tau_U                         0.010000
kappa_d                       5.000000
kappa                         5.000000
alpha_L                       3.000000
alpha_U                       3.000000
rho                           0.500000
c_d                           10.000000
c_h                           2.000000
N                             -1
verbose                       info
rf_par                        1
reorder                       Off
spd  

With the preconditioenr successfully been built, let's print out some basic information

In [6]:
print("M levels are {}, with nnz {}".format(M.levels, M.nnz))

M levels are 2, with nnz 114848


Alternatively, one can use the following codes:

```python
M = HIF()
params=Params()
params.tau = 1e-2  # equiv. to params["tau_L"]=params["tau_U"]=1e-2
params.kappa = 5.0
params.alpha = 3.0
M.factorize(A, params=params)
```
or
```python
M = HIF()
M.factorize(A,
    tau_L=0.01,
    tau_U=0.01,
    kappa=5.0,
    kappa_d=5.0,
    alpha_L=3.0,
    alpha_U=3.0,
)
```

## Apply HIF ##
We now consider applying `M` in triangular solve and matrix-vector multiplication two modes.

In [8]:
x = M.apply(b)
err = M.apply(x, op="M") - b
print("norm2(err)/norm2(b) =", np.linalg.norm(err)/np.linalg.norm(b))

norm2(err)/norm2(b) = 1.439076582138997e-17


In [9]:
# Tranpose
x = M.apply(b, op="SH")
err = M.apply(x, op="MH") - b
print("norm2(err)/norm2(b) =", np.linalg.norm(err)/np.linalg.norm(b))

norm2(err)/norm2(b) = 1.4514835137900503e-17
